In [2]:
import pandas as pd
import numpy as np

#### Cossim with different SVD vectors

In [4]:
df_cossim = pd.read_csv("/data/kebl6672/dpo-toxic-general/toxicity/gpt2_svd_all_neuron_cossims.csv")

In [12]:
# df1 = pd.read_csv("/data/kebl6672/dpo-toxic-general/toxicity/gpt2_svd_0_neuron_cossims.csv")
# df1 = df1.rename(columns={'cosine_similarity': 'svd_0_cossim'})
# df2 = pd.read_csv("/data/kebl6672/dpo-toxic-general/toxicity/gpt2_svd_1_neuron_cossims.csv")
# df2 = df2.rename(columns={'cosine_similarity': 'svd_1_cossim'})
# df3 = pd.read_csv("/data/kebl6672/dpo-toxic-general/toxicity/gpt2_svd_2_neuron_cossims.csv")
# df3 = df3.rename(columns={'cosine_similarity': 'svd_2_cossim'})

# merge_cols = ['layer_idx', 'neuron_idx']
# df = pd.merge(df1, df2, on=merge_cols, how="inner")
# df = pd.merge(df, df3, on=merge_cols, how="inner")

# df.to_csv('/data/kebl6672/dpo-toxic-general/toxicity/gpt2_svd_all_neuron_cossims.csv')

# df.head()

,layer_idx,neuron_idx,svd_0_cossim,svd_1_cossim,svd_2_cossim
0,0,0,0.014061,0.008812,-0.001678
1,0,1,0.029388,-0.031494,0.038727
2,0,2,-0.063171,0.018188,0.053436
3,0,3,-0.006809,-0.016769,0.064026
4,0,4,-0.005581,-0.025803,0.000685


In [5]:
cols = ['svd_0_cossim', 'svd_1_cossim', 'svd_2_cossim']
signs = df_cossim[cols].apply(np.sign)
same_sign = (signs.nunique(axis=1) == 1) # boolean df
misaligned_counts = (~same_sign).sum()
print(misaligned_counts/df_cossim.shape[0])

0.7468363444010416


#### Projection change after DPO for different SVD vectors

In [3]:
df_pt = pd.read_csv("/data/kebl6672/dpo-toxic-general/toxicity/gpt2_svd_all_pt_neuron_projections.csv")
df_dpo = pd.read_csv("/data/kebl6672/dpo-toxic-general/toxicity/gpt2_svd_all_dpo_neuron_projections.csv")
merge_cols = ['layer_idx', 'neuron_idx']
df = pd.merge(df_pt, df_dpo, on=merge_cols, how="inner")
df.head()

,Unnamed: 0_x,layer_idx,neuron_idx,0_pt_projection_value,0_pt_activation_value,1_pt_projection_value,1_pt_activation_value,2_pt_projection_value,2_pt_activation_value,Unnamed: 0_y,0_dpo_projection_value,0_dpo_activation_value,1_dpo_projection_value,1_dpo_activation_value,2_dpo_projection_value,2_dpo_activation_value
0,0,1,751,0.027710,-0.159912,-0.019760,-0.159912,-0.026215,-0.159912,0,0.027710,-0.160278,-0.019806,-0.160278,-0.026306,-0.160278
1,1,6,1095,0.003614,-0.043579,0.004757,-0.043579,-0.000522,-0.043579,1,0.004185,-0.050995,0.005566,-0.050995,-0.000607,-0.050995
2,2,4,1022,-0.002802,-0.058746,0.003963,-0.058746,0.001796,-0.058746,2,-0.002716,-0.056458,0.003811,-0.056458,0.001731,-0.056458
3,3,9,1366,0.011185,-0.100769,-0.017914,-0.100769,-0.010292,-0.100769,3,0.010956,-0.099670,-0.017685,-0.099670,-0.010185,-0.099670
4,4,0,3532,0.003799,-0.049347,-0.001810,-0.049347,0.000883,-0.049347,4,0.003706,-0.048126,-0.001767,-0.048126,0.000859,-0.048126


In [6]:
df['0_projection_diff'] = df['0_dpo_projection_value'] - df['0_pt_projection_value']
df['1_projection_diff'] = df['1_dpo_projection_value'] - df['1_pt_projection_value']
df['2_projection_diff'] = df['2_dpo_projection_value'] - df['2_pt_projection_value']

In [7]:
cols = ['layer_idx', 'neuron_idx', '0_projection_diff', '1_projection_diff', '2_projection_diff']
df_proj_diff = df[cols]

In [8]:
df_proj_diff.head()

,layer_idx,neuron_idx,0_projection_diff,1_projection_diff,2_projection_diff
0,1,751,0.000000,-0.000046,-0.000092
1,6,1095,0.000570,0.000809,-0.000085
2,4,1022,0.000086,-0.000153,-0.000065
3,9,1366,-0.000229,0.000229,0.000107
4,0,3532,-0.000093,0.000043,-0.000024


In [9]:
cols = ['0_projection_diff', '1_projection_diff', '2_projection_diff']
signs = df_proj_diff[cols].apply(np.sign)
same_sign = (signs.nunique(axis=1) == 1)
misaligned_counts = (~same_sign).sum()
print(misaligned_counts/df.shape[0])

0.7429097493489584


In [10]:
merge_cols = ['layer_idx', 'neuron_idx']
df_merge = pd.merge(df_proj_diff, df_cossim, on=merge_cols, how="inner")
df_merge

,layer_idx,neuron_idx,0_projection_diff,1_projection_diff,2_projection_diff,Unnamed: 0,svd_0_cossim,svd_1_cossim,svd_2_cossim
0,1,751,0.000000,-0.000046,-0.000092,4847,-0.068542,0.048859,0.064880
1,6,1095,0.000570,0.000809,-0.000085,25671,-0.033020,-0.043427,0.004768
2,4,1022,0.000086,-0.000153,-0.000065,17406,0.018524,-0.026215,-0.011864
3,9,1366,-0.000229,0.000229,0.000107,38230,-0.045197,0.072388,0.041534
4,0,3532,-0.000093,0.000043,-0.000024,3532,-0.029388,0.014023,-0.006836
...,...,...,...,...,...,...,...,...,...
98299,1,3784,-0.000717,-0.000156,0.000202,7880,0.067871,0.015869,-0.020630
98300,4,4055,-0.000298,0.000042,-0.000061,20439,-0.044586,0.005756,-0.012383
98301,7,1320,0.000255,0.003922,0.001080,29992,0.007355,0.096252,0.026566
98302,0,149,-0.000020,-0.000351,-0.000122,149,-0.001242,-0.065430,-0.022156


In [11]:
cols = ['svd_0_cossim', 'svd_1_cossim', 'svd_2_cossim']
cossim_signs = df_merge[cols].apply(np.sign)
same_cossim_sign = (cossim_signs.nunique(axis=1) == 1)
misaligned_cossim_cols = ~same_cossim_sign

cols = ['0_projection_diff', '1_projection_diff', '2_projection_diff']
proj_diff_signs = df_merge[cols].apply(np.sign)
same_proj_diff_sign = (proj_diff_signs.nunique(axis=1) == 1)
misaligned_proj_diff_cols = ~same_proj_diff_sign

# misaligned_counts = (~same_sign).sum()
# print(misaligned_counts/df.shape[0])
misaligned_cossim_cols.head()

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [12]:
misaligned_cossim_cols.head()

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [13]:
both_misaligned = misaligned_cossim_cols & misaligned_proj_diff_cols
num_both_misaligned = both_misaligned.sum()
total_rows = len(df_merge)

percentage = (num_both_misaligned / total_rows) * 100
print(percentage)


71.5301513671875


In [14]:
one_misaligned = ~misaligned_cossim_cols & misaligned_proj_diff_cols
num_one_misaligned = one_misaligned.sum()
total_rows = len(df_merge)

percentage = (num_one_misaligned / total_rows) * 100
print(percentage)

2.760823567708333


In [15]:
one_misaligned = ~misaligned_cossim_cols & misaligned_proj_diff_cols
num_one_misaligned = one_misaligned.sum()
total_rows = len(df_merge)

percentage = (num_one_misaligned / total_rows) * 100
print(percentage)

2.760823567708333


In [16]:
both_aligned = ~misaligned_cossim_cols & ~misaligned_proj_diff_cols
num_both_aligned = both_aligned.sum()
total_rows = len(df_merge)

percentage = (num_both_aligned / total_rows) * 100
print(percentage)

22.5555419921875
